# Récupération des données avec l'API Meteo France

La récupération des données à partir d'une API (Interface de Programmation d'Applications) est un processus clé dans l'interaction entre différentes applications ou services. Cette méthode permet d'accéder directement à des informations actualisées et fiables, en automatisant leur collecte grâce à des requêtes programmatiques.
Dans le cadre de ce projet, les données météorologiques ont été récupérées via l'API de Météo-France qui permet d'intégrer des données telles que les prévisions, les observations, les alertes.

Pour Débuter, nous récupérons la liste de toutes les stations météos disponibles. 
Une station météo permet mesurer et enregistrer divers paramètres atmosphériques afin de fournir des informations sur les conditions météorologiques locales.

In [2]:
#on récupère la liste de toutes les stations avec l'API ainsi que les coordonées telles que la longitude et la latitude
import fonctions as f
client=f.Client()

station=client.request('GET', 'https://public-api.meteofrance.fr/public/DPObs/v1/liste-stations', verify=False)

dataStation = f.StringIO(station.text)
print(dataStation)
df = f.pd.read_csv(dataStation,sep=';')
df = df.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'})
print(df)


      Id_station   Id_omm                Nom_usuel        lat         lon  \
0        1014002      NaN                   ARBENT  46.278167    5.669000   
1        1027003      NaN               BALAN_AERO  45.833000    5.106667   
2        1034004      NaN                   BELLEY  45.769333    5.688000   
3        1064001      NaN                 VERIZIEU  45.777167    5.487167   
4        1071001      NaN                    CESSY  46.310333    6.080333   
...          ...      ...                      ...        ...         ...   
2121    98832004  91588.0            MTGNE SOURCES -22.143833  166.593167   
2122    98832005      NaN                   OUINNE -21.984000  166.680500   
2123    98832006      NaN          RIVIERE BLANCHE -22.132667  166.726333   
2124    98832101      NaN  GORO_ANCIENNE_PEPINIERE -22.269167  166.967500   
2125    98833002      NaN                      MEA -21.455500  165.767333   

      Altitude Date_ouverture    Pack  
0          534     2003-10-01  RADO

In [4]:
# On crée un GeoDataFrame contenant des géométries de type Point à partir de colonnes de coordonnées géographiques (longitude et latitude) présentes dans un DataFrame

gdf = f.gpd.GeoDataFrame(
    df, geometry=f.gpd.points_from_xy(df.lon, df.lat), crs=4326
)
gdf

,Id_station,Id_omm,Nom_usuel,lat,lon,Altitude,Date_ouverture,Pack,geometry
0,1014002,NaN,ARBENT,46.278167,5.669000,534,2003-10-01,RADOME,POINT (5.669 46.27817)
1,1027003,NaN,BALAN_AERO,45.833000,5.106667,196,2014-05-26,ETENDU,POINT (5.10667 45.833)
2,1034004,NaN,BELLEY,45.769333,5.688000,330,2001-09-13,RADOME,POINT (5.688 45.76933)
3,1064001,NaN,VERIZIEU,45.777167,5.487167,281,2015-01-01,ETENDU,POINT (5.48717 45.77717)
4,1071001,NaN,CESSY,46.310333,6.080333,507,2002-05-01,RADOME,POINT (6.08033 46.31033)
...,...,...,...,...,...,...,...,...,...
2121,98832004,91588.0,MTGNE SOURCES,-22.143833,166.593167,773,1989-08-01,ETENDU,POINT (166.59317 -22.14383)
2122,98832005,NaN,OUINNE,-21.984000,166.680500,54,1974-01-01,ETENDU,POINT (166.6805 -21.984)
2123,98832006,NaN,RIVIERE BLANCHE,-22.132667,166.726333,171,2000-11-01,ETENDU,POINT (166.72633 -22.13267)
2124,98832101,NaN,GORO_ANCIENNE_PEPINIERE,-22.269167,166.967500,298,1995-01-01,ETENDU,POINT (166.9675 -22.26917)


In [ ]:
#On représente sur une carte toutes les stations de l'API de météo FRANCE
m2=f.interactive_map_dpe(gdf)
m2

Dans la suite notre étude portera sur le département 01 en France correspondant à l'Ain, situé dans la région Auvergne-Rhône-Alpes, au sud-est du pays.
En effet ce choix s'explique par le fait que ce département est sujet à des périodes de canicules et à des catastrophes naturelles.

In [ ]:
#On récupere la liste des stations climatologiques avec les mesures à une fréquence de 1 journée du départementb de l'AIN
client = f.Client()
client.session.headers.update({'Accept': 'application/json'})
response = client.request('GET', 'https://public-api.meteofrance.fr/public/DPClim/v1/liste-stations/infrahoraire-6m?id-departement=01', verify=False)
df=f.pd.DataFrame(response.json())
print(df.head())


         id         nom  posteOuvert  typePoste       lon        lat  alt  \
0  01014002      ARBENT         True          1  5.669000  46.278167  534   
1  01033002  BELLEGARDE        False          2  5.814167  46.086500  350   
2  01034004      BELLEY         True          1  5.688000  45.769333  330   
3  01064001    VERIZIEU         True          2  5.487167  45.777167  281   
4  01071001       CESSY         True          1  6.080333  46.310333  507   

   postePublic  
0         True  
1         True  
2         True  
3         True  
4         True  


In [7]:
gdf = f.gpd.GeoDataFrame(
    df, geometry=f.gpd.points_from_xy(df.lon, df.lat), crs=4326
)
gdf



,id,nom,posteOuvert,typePoste,lon,lat,alt,postePublic,geometry
0,91027002,ORLY,True,0,2.397000,48.718000,86,True,POINT (2.397 48.718)
1,91069001,BOIGNEVILLE,False,2,2.381167,48.325500,115,True,POINT (2.38117 48.3255)
2,91103001,BRETIGNY_SAPC,True,2,2.326000,48.600500,78,True,POINT (2.326 48.6005)
3,91184001,COURDIMANCHE,True,1,2.374333,48.421833,71,True,POINT (2.37433 48.42183)
4,91200002,DOURDAN,True,1,1.993000,48.526167,116,True,POINT (1.993 48.52617)
5,91275001,GOMETZ-LE-CHAT.,True,2,2.135833,48.676833,159,True,POINT (2.13583 48.67683)
6,91405002,MILLY-LA-FORET,True,2,2.431833,48.382167,109,True,POINT (2.43183 48.38217)
7,91679001,VILLIERS-LE-BAC,False,2,2.111500,48.730333,147,True,POINT (2.1115 48.73033)
8,91679002,VILLIERS-LE-BACLE2,False,2,2.116167,48.735333,161,True,POINT (2.11617 48.73533)


In [ ]:
#Représentation des stations météos de l'AIN 
m=f.interactive_map_dpe(gdf)
m


Pour avoir accès a nos données nous devons tout d'abord générer un bon de commande.Le principe du "bon de commande" pour accéder à notre API consiste à envoyer une requête structurée qui précise ce que nous souhaitons obtenir, via des paramètres comme la date de début et de fin, une URL spécifique et une clé API ou un token pour l'authentification. Une fois la requête envoyée , le serveur vérifie nos droits d'accès et retourne une réponse avec les données demandées. Ce processus garantit clarté, traçabilité et flexibilité pour interagir avec l'API.

In [ ]:
#On créer une fonction qui permet de générer une base contentant les données météoroliques en fonction des dates de début et et de fin que nous souhaitons étudier
def gener_data(date_deb, date_fin):
    client=f.Client()
    datameteo=client.request('GET',"https://public-api.meteofrance.fr/public/DPClim/v1/commande/fichier?id-cmde={}".format(f.gener_idcommande(date_deb,date_fin)),verify=False)
    print(datameteo)
    data = f.StringIO(datameteo.text)
    print(data)
    f.time.sleep(5)
    return f.pd.read_csv(data,sep=';')

Notre API permet de récupérer des données sur une période de 1 mois uniquement. Nous créons donc des matrices contenant les dates des années 2022 à 2024


In [ ]:
matrix2022=f.generate_month_matrix(2022)
matrix2023=f.generate_month_matrix(2023)
matrix2024=f.generate_month_matrix(2024)
matrix2024=matrix2024[:10]

In [ ]:
#Apercu de la matrice2024 contentant les dates du mois de janvier à Octobre
matrix2024

[(datetime.date(2024, 1, 5), datetime.date(2024, 1, 31)),
 (datetime.date(2024, 2, 1), datetime.date(2024, 2, 28)),
 (datetime.date(2024, 3, 1), datetime.date(2024, 3, 31)),
 (datetime.date(2024, 4, 1), datetime.date(2024, 4, 30)),
 (datetime.date(2024, 5, 1), datetime.date(2024, 5, 31)),
 (datetime.date(2024, 6, 1), datetime.date(2024, 6, 30)),
 (datetime.date(2024, 7, 1), datetime.date(2024, 7, 31)),
 (datetime.date(2024, 8, 1), datetime.date(2024, 8, 31)),
 (datetime.date(2024, 9, 1), datetime.date(2024, 9, 30)),
 (datetime.date(2024, 10, 1), datetime.date(2024, 10, 31))]

In [ ]:
# Récupération des données pour l'année 2022

data2022=f.pd.DataFrame()
for i in range(len(matrix2022)):
    data2022=f.pd.concat([data2022,gener_data(matrix2022[i][0],matrix2022[i][1])], ignore_index=True)

<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>


In [ ]:
# Récupération des données pour l'année 2023
data2023=f.pd.DataFrame()
for i in range(len(matrix2023)):
    data2023=f.pd.concat([data2023,gener_data(matrix2023[i][0],matrix2023[i][1])], ignore_index=True)

<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>


In [ ]:
# Récupération des données pour l'année 2024
data2024=f.pd.DataFrame()
for i in range(len(matrix2024)):
    data2024=f.pd.concat([data2024,gener_data(matrix2024[i][0],matrix2024[i][1])], ignore_index=True)

<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>
<Response [201]>


In [ ]:
#On procède à la concaténation des bases 2022, 2023 et 2024

datameteo=f.pd.concat([data2022,data2023], ignore_index=True)
datameteo=f.pd.concat([datameteo,data2024], ignore_index=True)


In [19]:
# On convertit la base de données existante en fichier cvs 
datameteo.to_csv('Données/data.csv', index=False)